In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from numpy.lib.npyio import itertools
from sklearn.model_selection import train_test_split

ratings = pd.read_csv('normalizedreviews.csv',
                      sep=';',  usecols=['movieId', 'userId', 'rating', 'review_content'])
movies = pd.read_csv('movies.csv',
                     sep=';',  usecols=['rotten_tomatoes_link', 'movie_title', 'movie_info',  'genres', 'directors', 'actors', 'original_release_date', 'tomatometer_rating', 'audience_rating'])



def contentbased(user,movies,ratings):
        movies = movies[movies['rotten_tomatoes_link'].isin(
                ratings['movieId'].to_numpy())]
        movies.dropna(subset=['movie_info'], inplace=True)
        ratings.dropna(axis=0, subset=['rating'], inplace=True)
        ratings.drop_duplicates(subset=['movieId', 'userId'], inplace=True)
        userLID = ratings['userId'].unique()

        tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3),
                        min_df=0, stop_words='english')
        tfidf_matrix_item = tf.fit_transform(movies['movie_info'])
        sum=0


        userRate = ratings[ratings['userId'] == user]
        relRating = userRate[userRate['rating'] >= 4]
        userM = movies[movies['rotten_tomatoes_link'].isin(relRating['movieId'])]
        if (userM.shape[0]>=4 and relRating.shape[0]>=4):                
                userMrel, userMT = train_test_split(userM,test_size=0.4, shuffle=True)
                userMT = userMT['movieId'].to_numpy()
                        
                featureMat = pd.DataFrame(tfidf_matrix_item.todense(),
                                                columns=tf.get_feature_names_out(),
                                                index=movies.movieId)
                featureMatU = featureMat[featureMat.index.isin(userMrel['movieId'])]
                featureMatU = (pd.DataFrame((featureMatU.mean()),
                                                columns=['similarity'])).transpose()
                        
                        
                        
                cosine_sim = cosine_similarity(featureMatU, tfidf_matrix_item)
                cosine_sim_df = pd.DataFrame(cosine_sim.T, index=movies['movieId'], columns=['similarity'])
                




"""       
result = cosine_sim_df.sort_values(by='similarity', ascending=False).head(10)
n = 0
rslt= result.index.to_numpy()
for movie in rslt:
        if movie in userMT:
                n+=1
        sum=sum+n
sum= sum/10
sum= sum/len(userLID)
print(sum)
"""
